In [12]:
import wandb
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score,roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize
import pandas as pd

In [13]:
digital_class=pd.read_csv(r"C:\Users\Russhi\Desktop\BOOTCAMP\Module3_0_ProyectoFinal\digital_class.csv", sep=',')

In [14]:
from sklearn.preprocessing import OneHotEncoder
encoder= OneHotEncoder(drop='first', sparse_output=False)
columnas=['Disponibilidad', 'Estado', 'Envios','Categoria Principal','secondary category']
categorical_data = digital_class[columnas]
encoded_categorical_data = pd.DataFrame(encoder.fit_transform(categorical_data))
encoded_categorical_data.columns = encoder.get_feature_names_out(columnas)
digital_class = digital_class.drop(['Disponibilidad', 'Estado', 'Envios','Categoria Principal','secondary category'], axis=1)
encoded_data = pd.concat([digital_class, encoded_categorical_data], axis=1)

In [15]:
encoded_data['Tienda Precio']=encoded_data['Tienda Precio'].map({'bestbuy':0,'walmart':1,'bhphotovideo':2,'ebay':3})

In [16]:
X = encoded_data.drop(columns = ['Tienda Precio'], axis=1).copy() 
y = encoded_data['Tienda Precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=357)

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
hyperparameters ={
    'learning_rate': [0.01,0.15,0.5], 'max_depth': [2,3,5], 'n_estimators':[100,170,200], 
    'loss_functions': ['deviance'], 'subsamples': [0.4,0.8,1], 'min_samples_split': [2,4,6],
    'min_samples_leafs':[1,2,3]}

In [ ]:
for lr in hyperparameters['learning_rate']:
    for max_depth in hyperparameters['max_depth']:
        for n_estimator in hyperparameters['n_estimators']:
            for loss_function in hyperparameters['loss_functions']:
                for subsample in hyperparameters['subsamples']:
                    for min_samples_split in hyperparameters['min_samples_split']:
                        for min_samples_leaf in hyperparameters['min_samples_leafs']:

                            # Iniciar wandb
                            experiment_name = f"gbm_lr{lr}_depth{max_depth}_est{n_estimator}_loss{loss_function}_subsample{subsample}_minsplit{min_samples_split}_minleaf{min_samples_leaf}"
                            wandb.init(project="project_digital4", name=experiment_name, config=hyperparameters)
                            # Entrenamiento
                            clf = GradientBoostingClassifier(learning_rate=lr, max_depth=max_depth, n_estimators=n_estimator,
                                                             loss=loss_function, subsample=subsample, min_samples_split=min_samples_split,
                                                             min_samples_leaf=min_samples_leaf, random_state=888, validation_fraction=0.1,
                                                             n_iter_no_change=5, tol=0.01)
                            clf.fit(X_train, y_train)
                            # Predicción
                            y_pred = clf.predict(X_test)
                            y_pred_proba = clf.predict_proba(X_test)
                            

                            #Registro de HP
                            wandb.config.learning_rate = lr
                            wandb.config.max_depth = max_depth
                            wandb.config.n_estimators = n_estimator
                            wandb.config.loss_function = loss_function
                            wandb.config.subsample = subsample
                            wandb.config.min_samples_split = min_samples_split
                            wandb.config.min_samples_leaf = min_samples_leaf

                            # Cálculo de métricas
                            accuracy = accuracy_score(y_test, y_pred)
                            f1_macro = f1_score(y_test, y_pred, average="macro")
                            y_test_bin = label_binarize(y_test, classes=np.unique(y))
                            y_pred_bin = y_pred_proba.reshape(-1, len(np.unique(y)))
                            roc_auc_macro = roc_auc_score(y_test_bin, y_pred_bin, average="macro", multi_class="ovr")
                            # Registrar las métricas en wandb
                            wandb.log({"accuracy": accuracy, "f1_macro": f1_macro, "roc_auc_macro": roc_auc_macro,
                                       "validation_score": clf.train_score_[-1]})
                            # Finalizamos el experimento
                            wandb.finish()

link = https://wandb.ai/alba-m-boga/project_digital4/reports/Modelo-de-clasificaci-n-de-tiendas--Vmlldzo0ODg2ODg4